# Summarization model
Take a pre-trained model **MBARTRuSumGazeta** = https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta
The model is pre-trained on "Dataset for Automatic Summarization of Russian News" - https://arxiv.org/abs/2006.11063


# Setup

In [1]:
!pip install transformers sentencepiece

     |████████████████████████████████| 4.2 MB 27.4 MB/s 
     |████████████████████████████████| 1.2 MB 59.0 MB/s 
     |████████████████████████████████| 6.6 MB 58.1 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Import

In [2]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [35]:
model_mbart = MBartForConditionalGeneration.from_pretrained(model_name)
model_mbart.to("cuda")

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

# Class

In [40]:
class Summarization():
  def __init__(self, model):
    self.model = model


  def getSummary(self, text):
    self.input_ids = False
    self.output_ids = False
    self._input(text)
    return self._generate()


  def _input(self, text):
    self.input_ids = tokenizer(
      [text],
      max_length=600,
      truncation=True,
      return_tensors="pt",
    )["input_ids"].to("cuda")


  def _generate(self):
    self.output_ids = self.model.generate(
      input_ids = self.input_ids,
      no_repeat_ngram_size=4
    )[0]
    summary = tokenizer.decode(self.output_ids, skip_special_tokens=True)
    return summary

In [41]:
model = Summarization(model_mbart)

## Example_1

In [44]:
text = '''
Положительный отзыв Тинькофф Практичность и удобство
Здравствуйте, хотелось бы написать отзыв о Тинькофф Банке, около года назад начал 
пользоваться услугами Тинькофф (кредитная, дебетовая карты + тинькофф инвестиции), 
мне прям очень понравилось, тинькофф приложение работает более чем отлично, никогда 
не зависает, не лагает, за переводы еще ни разу не видел комиссию, в отличие от 
других банков, очень вкусный кешбек, недавно не хватало некоторой суммы денег на 
новое игровое кресло, воспользовался услугами кредитной карты и 
остался очень доволен. Выбрал я изначально этот банк именно из-за кешбека, узнал 
о банке благодаря лимитированной коллекции ALL Games.Плюсом к этому убедило меня 
взять именно этот продукт полная отсутствие платы за обслуживание карты, да еще 
и копеечку от остатка на счете можно заработать, как говорится "мелочь, а приятно" 
Ставлю 5
'''

In [45]:
model.getSummary(text)

'Взял кредит в Тинькофф Банке из-за того, что не хватало денег на новое игровое кресло.'

## Example_2

In [46]:
text = '''
Рассказ о том, как я осталась в чужом городе без копейки в кармане. Привет Совкомбанку!
Ровно неделю назад я отправилась в командировку в чудесный город Санкт-Петербург. 
Ничего не предвещало беды. Билеты уже были оплачены, а вот отель надо было оплатить 
в день выселения. Командировка подошла к концу, медленно, но верно подошел как раз 
последний день, сдать и оплатить номер мне надо было до 12:00. В это же день я получаю 
прекрасное оповещение, что моя карта заблокирована и приложение заодно. Я сначала 
подумала, что это какая-то ошибка: кредитов в Совкомбанке у меня нет и не было, 
у Халвы стоит лимит 0,10, т.е. влезть в заемные средства я не могла. Звонок в тех 
поддержку: мои данные проверяют. Сотрудник отвечает: "Нет, ошибки нет, банку не 
понятны денежные поступления, которые приходят на карту". Объясняю, что я в другом 
городе, в командировке, мне деньги переводит супруг. Ответ: "мы очень вам сочувствуем, 
но ничем помочь не можем, ждем вас в офисе банка с 2 НДФЛ, разблокировать карту 
можно только в родном городе после приезда". Какая 2 НДФЛ в чужом городе, ребята? 
Объясняю все по второму кругу. Ответ: "ну, мы очень вам сочувствуем, но ничем помочь 
не можем, ждем вас в вашем родном городе в офисе банка с 2 НДФЛ". Что мне делать?.. 
Я без наличных, без каких-либо вообще денег, надо расплатиться за отель, самолет 
только вечером, надо что-то еще кушать, пить, доехать до аэропорта, наконец. 
В шоковом состоянии нахожусь, сижу в номере просто с этой картой в руке, смотрю 
в одну точку. Наличку я много лет уже не снимаю, езжу везде с картой, удобно и 
практично. Не помню, когда я вообще в последний раз платила наличными, тем более 
карты принимают практически везде. Теперь, видимо, буду умнее. Меня выручила подруга, 
которая проживала в Петербурге, она приехала, все оплатила. Уже в родном городе 
я пришла в офис, у меня был 1% надежды, что все-таки это ошибка. Финансовый консультант 
уточняет: "это ваша зарплатная карта?"

нет, не зарплатная
а, ну, тогда понятно, банку не понятны денежные поступления, которые приходят на 
карту... Ребята, вы там совсем что ли того? А зачем мне еще карта банка, если на 
нее не могут прийти денежные поступления??? Где в законе установлен регламент, что 
на карту банка может поступать ТОЛЬКО заработная плата? И я понимаю, если бы обороты 
у меня были по миллиону или хотя бы по 500 тысяч в месяц... Мне распечатали мои 
транзакции, там суммы то смешные к переводу: по 3 тысячи, по 5 тысяч, по 10 тысяч. 
Оборот в месяц не превышает 50 тысяч. Неужели это та самая сумма, которая тревожит 
банк??? Не смешите меня! Обратите внимание на карты, по которым проходят миллионы 
в месяц. А вас смущает моя? Сейчас просто ради принципа я восстановлю Халву с 2 
НДФЛ, чтобы потом закрыть ее насовсем. Уже рассказала своим близким, друзьям и 
знакомым, чтобы в этот банк ни ногой. Раньше была в восторге от Халвы, всем ее
 рекламировала. Сейчас после моих новостей, уже двое друзей закрыли свои счета и 
 карты, потому что много путешествуют и так же переводят друг другу деньги с карты 
 на карты. А остаться без копейки в кармане в чужом городе, как я, никто не хочет. 
 Мне 30 с небольшим лет, у меня было много карт, и много платежей и переводов по 
 ним проходило. Но блокировать карту и приложение из-за таких смешных транзакций - 
 это не достойно банка, который претендует на звание народного банка. Совкомбанк, 
 прощай однозначно, к вам больше ни ногой никогда!
'''

In [48]:
model.getSummary(text)

'Совкомбанк заблокировал мою зарплатную карту в Санкт-Петербурге. Позвонив в тех поддержку, я узнала, что карта заблокирована, мне не понятны денежные поступления, которые приходят на карту.'

## Example_3

In [49]:
text = '''
Карта халва
Воспользовался программой кредитный доктор, все понравилось , спасибо! 
Ждём результатов программы потом дополню отзыв
'''

In [50]:
model.getSummary(text)

'В кредитном докторе мне все понравилось, спасибо! Ждём результатов программы потом дополню отзыв.'